In [ ]:
import sys  
import os
sys.path.insert(0, '/'.join([*os.getcwd().split('/')[:-1], 'Utils']))

import notifications as n

import requests
from bs4 import BeautifulSoup
import json
import os

import time

import pandas as pd
pd.set_option('display.max_columns', None)

from selenium import webdriver

from dotenv import load_dotenv

In [ ]:
load_dotenv(override = True)

In [ ]:
def get_body(site_url):
    headers = {'Accept-Encoding': 'identity'}
    site_source = requests.get(site_url, headers=headers)
    content = BeautifulSoup(site_source.content).body
    return content

In [ ]:
def check_halo(content):
    b = content.find(attrs = {'type': 'button', 'class': 'BundleBuilderHeader-module__checkoutButton___3UyEq'})
    if 'disabled' in b.attrs:
        return False
    else:
        True

In [ ]:
def check_microsoft(content, keyword):
    buys = content.find(attrs = {'class':"buy-group"})
    if buys != None:
        lst = buys.findChildren(name='span')

    if any([keyword in i.text for i in lst]) != True:
        return True
    else:
        return False

In [ ]:
def check_ms_inventory():
    response = requests.post(
    url = 'https://inv.mp.microsoft.com/v2.0/inventory/NL',
    data = """[
        {"skuId":"RRT-00009","distributorId":"9000000013"},
        {"skuId":"RRT-00007","distributorId":"9000000013"},
        {"skuId":"RRT-00008","distributorId":"9000000013"},
        {"skuId":"RRT-00010","distributorId":"9000000013"}]""",
    params = {'MS-CorrelationId':'98e7083c-4bfb-4715-b3cb-4b423d01c9cf',
                      'MS-RequestId': '98e7083c-4bfb-4715-b3cb-4b423d01c9cf',
                      'mode':'continueOnError'},
     headers = {'accept': '*/*',
         'accept-encoding': 'gzip, deflate, br',
         'accept-language': 'en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7',
         'content-length': '817',
         'content-type': 'application/json',
         'dnt': '1',
         'origin': 'https://www.microsoft.com',
         'referer': 'https://www.microsoft.com/',
         'sec-ch-ua': '"Google Chrome";v="93", " Not;A Brand";v="99", "Chromium";v="93"',
         'sec-ch-ua-mobile': '?0',
         'sec-ch-ua-platform': '"Windows"',
         'sec-fetch-dest': 'empty',
         'sec-fetch-mode': 'cors',
         'sec-fetch-site': 'same-site',
         'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36'}
                            )
    d = eval(response.text)
    df = pd.DataFrame(d)
    df_availables =  pd.json_normalize(df['availabilities'])
    df_availables['inStock'] = df['inStock'].apply(eval)
    col = 'productDescription'
    final_d = df_availables[[col, 'inStock']].set_index(col)['inStock'].to_dict()
    return final_d

In [ ]:
def check_coolblue():
    url = 'https://www.coolblue.nl/en/product/867421/xbox-series-x.html'
    request = BeautifulSoup(requests.get(url).content)
    lst = request.find_all(attrs = {'type': 'submit', 'class': 'js-add-to-cart-button'}, name = 'button')

    if len(lst)>0:
        return True
    else:
        return False

In [ ]:
def check_bcc():
    url = 'https://www.bcc.nl/gaming/xbox/xbox-series-console/microsoft-gameconsole-xbox-series-x/293589'
#     url = 'https://www.bcc.nl/gaming/xbox/xbox-gamepass-abonnement/xbox-game-pass-console-lidmaatschap-3-maanden-direct-download/292764'
    request = BeautifulSoup(requests.get(url).content)

    lst = request.find_all(attrs = {'class': 'productoffer__orderbtn'})

    if len(lst)>0:
        return True
    else:
        return False

In [ ]:
class MediaMarkt:
    '''MediaMarkt uses javascript to check online availability. 
    Thus we have to let the page load in Selenium before getting the results.'''
    def __init__(self, driver=None):
        self._driver = driver
        
    def start_mediatmarkt_driver(self):
        driver = webdriver.Chrome()
        driver.get('https://www.mediamarkt.nl/nl/product/_microsoft-xbox-series-x-1667555.html#')
        return driver
    
    @property
    def driver(self):
        if self._driver == None:
            self._driver = self.start_mediatmarkt_driver()
            return self._driver
        else:
            return self._driver
    
    def check_mediamarkt(self):
        try:
            self.driver.refresh()
            online_nostock = self.driver.find_elements_by_class_name("online-nostock")
        except:
            self._driver = None
            online_nostock = self.driver.find_elements_by_class_name("online-nostock")

        if len(online_nostock) == 0:
            return True
        else:
            return False

In [ ]:
def check_bol():
    bol_url = 'https://www.bol.com/nl/nl/p/xbox-series-x-console/9300000003688723/'
    r = requests.get(bol_url)
    soup = BeautifulSoup(r.text)
    results = soup.findChildren(attrs = {'class': 'buy-block__title'})
    
    if results:
        return False
    else:
        return True

In [ ]:
def run_all_checks(mm: MediaMarkt):
    site='microsoft_xbox'
    site_urls = sites[site]['urls']
    keywords = sites[site]['keywords']

    availables = {}
    
    for site_url, keyword in zip(site_urls, keywords):
        content = get_body(site_url)
        available = check_microsoft(content, keyword)
        availables[site_url] = available

    site='microsoft_halo'

    for site_url in sites[site]:
        content = get_body(site_url)
        available = check_halo(content)
        availables[site_url] = available

    availables.update({'MS inventories':check_ms_inventory()})
    availables.update({'https://www.coolblue.nl/en/product/867421/xbox-series-x.html': check_coolblue()})
    availables.update({'https://www.bcc.nl/gaming/xbox/xbox-series-console/microsoft-gameconsole-xbox-series-x/293589':check_bcc()})
    availables.update({'https://www.mediamarkt.nl/nl/product/_microsoft-xbox-series-x-1667555.html#': mm.check_mediamarkt()})
    availables.update({'https://www.bol.com/nl/nl/p/xbox-series-x-console/9300000003688723': check_bol()})
    return availables

In [ ]:
# Gather Links
f = r"./xbox.json"
with open(f,  encoding='utf-8') as json_file:
    sites = json.load(json_file)

In [ ]:
# Google Chat webhook
xbox = os.environ['xbox']

In [ ]:
run = True
counter = 0
mm = MediaMarkt()
while run:
    try:
        availables = run_all_checks(mm)
        anystock = any(availables.values())
        counter+=1
        if anystock:
            available = {i:j for i, j in availables.items() if j == True}
            print(available)
            n.send_hangout(bot_message = available, webhook = xbox)
            run = False
            mm.driver.close()
            print(counter)
        else:
            time.sleep(3*60)
            continue
    except KeyboardInterrupt:
        mm.driver.close()
        break
    except Exception:
        print('skipped')
        time.sleep(3*60)
        continue
    